# Libraries

In [ ]:
import os, time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

# [UDF] How to reduce data file size
 - pd.read_pickle( ) & df.to_pickle( )
 - pickle: binary 타입으로 저장
 - csv/excel vs. pickle
   - csv/excel
     - better for 숫자, 단어로 이루어진 정형화된 데이터
     - MS Excel로 데이터를 다루고 싶을 때 이용
     - pickle보다 속도는 느리고 file size는 더 크다.
     - 데이터를 다시 불렀을 때, 저장했을 때 형태가 다를 수 있다.
   - pickle
     - work well with 장문 텍스트, 단문 텍스트 등의 비정형 데이터
     - work well with 리스트/데이터프레임 등을 포함한 데이터
     - python pandas를 이용해 데이터를 다루고 싶을 때 이용
     - Excel보다 속도는 빠르고 file size는 작다.
     - python 외의 다른 언어에서는 다룰 수 없다.
 - References
   - https://wikidocs.net/161041

In [3]:
import os
import time
import pandas as pd

def save_as_pickle(df, fpath):
    tic = time.time()
    df.to_pickle(os.path.join(fpath))
    toc = time.time()
    print('It took {:4f} seconds to save dataframe as pickle file.'.format(toc-tic))
    
def read_pickle(pkl_fpath):
    tic = time.time()
    df = pd.read_pickle(pkl_fpath)
    toc = time.time()
    print('It took {:4f} seconds to read the pickle file.'.format(toc-tic))

In [4]:
# usage example
df = pd.DataFrame({'a':[1, 'abc'], 'b':[3231, 'bdsf']})
fpath = './test914.pkl'

save_as_pickle(df, fpath)

It took 0.002321 seconds to save dataframe as pickle file.


In [6]:
# usage example
fpath = './test914.pkl'
read_pickle(fpath)

It took 0.001103 seconds to read the pickle file.


# Viz

In [ ]:
s = pd.Series([1,2,3,3,3,3,3,4,4,4,5]).value_counts()
width, 